# **저시력자를 위한 원화 화폐 분류**
---
- 본 과제는 UltraLytics YOLO v5 모델 사용을 권장합니다.
    - 본 파일의 목차는 UltraLytics YOLO v5에 맞게 작성되어 있습니다.
    - 다른 모델을 찾아서 사용하셔도 좋습니다.
    - 산출물이 잘 나오면 됩니다 : )
---

## 0.미션
---
- **과제 수행 목표**
    - 본 과제는 Object Detection 문제입니다.
    - Object Detection 문제로 접근하기 위해 **데이터셋 전처리**를 하셔야 합니다.
    - 데이터셋 : money_dataset.zip
        1. 데이터셋은 압축 파일로 제공됩니다.
        2. 압축 파일 안에는 화폐마다 폴더가 개별적으로 존재합니다.
        3. 폴더 안에는 화폐 이미지와 화폐 정보가 담긴 json 파일이 있습니다.
    - 여러분이 직접 촬영한 화폐 사진들을 탐지 과정에서 이용 해보세요.
    - 이미지에 화폐 하나만 나오게 촬영하는 것은 지양해주세요.
    - 다양한 방법으로 화폐를 촬영하고 결과를 확인해보세요.
        - ex 1) 화폐의 모든 종류를 한 이미지에 나오게 촬영
        - ex 2) 여러 화폐를 겹치게 하여 촬영
---
- **Key Point**
    1. 모델에 맞는 폴더 구조 확인
    2. 이미지 축소 비율에 맞춰 좌표값 변경
        - 좌표를 이미지 리사이즈한 비율로 변경
    3. 모델에 맞는 정보 추출/형식 변경
        - json 파일에서 정보 추출 및 모델 형식에 맞게 변경
    4. 화폐당 하나의 클래스로 변경
        - 총 8개 클래스
    5. 모델 선택 필요
---

## 1.환경설정

### (1) 구글 드라이브 연동, 데이터 다운로드
---
- 아래의 코드 셀을 반드시 실행시켜야 합니다.
---

In [1]:
# from google.colab import drive
# drive.mount('/content/drive/')

In [2]:
# !pip install gdown

### (2) 데이터셋 불러오기
---
- **세부요구사항**
    - 데이터셋 파일의 압축을 해제하세요.
---
- 예제 코드에서는 zipfile 모듈을 이용하였습니다.
    - [zipfile document](https://docs.python.org/3/library/zipfile.html#zipfile-objects)
    - 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
---

In [3]:
# import zipfile, gdown,os
# url ="https://drive.google.com/file/d/1k1tXDK35s6BsMTPGWSl5GVGNoPfC898X/view?usp=drive_link"
# file_name = "money_dataset.zip"
# output = "/content/drive/MyDrive/" + file_name # 변경 가능
# if not os.path.exists(output):
#     gdown.download(url=url, output=output, quiet=False, fuzzy=True)

In [4]:
# # 데이터셋 압축 파일 경로 : 유저별로 상이할 수 있음
# money_data = zipfile.ZipFile(output)

In [5]:
# # 데이터셋 압축 해제
# money_data.extractall('/content/Dataset/')

## 2.데이터 전처리

### (1) 폴더 구조 생성 및 파일 이동
---
- **세부요구사항**
    -  모델에서 요구하는 폴더 구조를 만들어야 합니다.
        - Hint : Image와 Label을 구분하는 폴더를 만들어 주세요
---
- 예제 코드에서는 glob, shutil 모듈을 이용하였습니다.
    - [glob document](https://docs.python.org/3/library/glob.html) | [shutil document](https://docs.python.org/3/library/shutil.html)
    - 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
---

In [6]:
# # 1.폴더 구조 만들기
# !mkdir ./content/Dataset/images;
# !mkdir ./content/Dataset/images/train; mkdir /content/Dataset/images/val

# !mkdir ./content/Dataset/labels;
# !mkdir ./content/Dataset/labels/train; mkdir /content/Dataset/labels/val

In [3]:
import os
import shutil

def clear_folder(folder_path):
    if os.path.exists(folder_path):
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.unlink(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
            except Exception as e:
                print(f'Failed to delete {file_path}. Reason: {e}')

    # 폴더를 다시 생성
    os.makedirs(folder_path, exist_ok=True)

# 사용 예시
clear_folder("./content/Dataset/images/train")
clear_folder("./content/Dataset/images/val")
clear_folder("./content/Dataset/labels/train")
clear_folder("./content/Dataset/labels/val")

In [4]:
# 2. Dataset metadata 입력
won_list = ['10', '50', '100', '500', '1000', '5000', '10000', '50000']
data_path = './content/Dataset/'

In [5]:
print(os.path.exists(data_path))

True


---
- 데이터를 Training set | Validation set으로 분할하세요.
    - 예시 : Training과 Validation은 8:2로 분리
- Hint : 이미지 데이터는 /images에, JSON 데이터는 /labels에 넣어주세요
    - 예시 : /dataset/images/train, /dataset/labels/train
    - 예제 코드에서는 glob, shutil 모듈을 이용하였습니다.
    - [glob document](https://docs.python.org/3/library/glob.html) | [shutil document](https://docs.python.org/3/library/shutil.html)

    ※ 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
    
---

In [6]:
# 3. 데이터를 Training set | Validation set으로 분할

In [7]:
# 모듈 불러오기
import glob
import shutil
import os

# 이미지와 JSON 파일이 저장되어 있는 디렉터리 목록
won_list = ['10', '50', '100', '500', '1000', '5000', '10000', '50000']
data_path = './content/Dataset/money_dataset/'

# 8:2 비율로 Training과 Validation을 분리합니다.
train_ratio = 0.8

# 각 폴더 내의 파일을 읽어와서 분류합니다.
for won_value in won_list:
    image_files = glob.glob(f'{data_path}{won_value}/*.jpg')
    json_files = glob.glob(f'{data_path}{won_value}/*.json')
    
    # 분할 지점 계산
    train_image_count = int(len(image_files) * train_ratio)
    train_json_count = int(len(json_files) * train_ratio)
    
    # 이미지 파일 분류
    for i, file_path in enumerate(image_files):
        if i < train_image_count:
            # Training 세트
            shutil.copy(file_path, f'./content/Dataset/images/train/{os.path.basename(file_path)}')
        else:
            # Validation 세트
            shutil.copy(file_path, f'./content/Dataset/images/val/{os.path.basename(file_path)}')

    # JSON 파일 분류
    for i, file_path in enumerate(json_files):
        if i < train_json_count:
            # Training 세트
            shutil.copy(file_path, f'./content/Dataset/labels/train/{os.path.basename(file_path)}')
        else:
            # Validation 세트
            shutil.copy(file_path, f'./content/Dataset/labels/val/{os.path.basename(file_path)}')

### (2) json에서 정보 추출
---
- **세부요구사항**
    - json 파일에서 필요한 정보를 추출하세요:
        - 위치 정보 : x1, x2, y1, y2
        - 박스 정보 : shape_type
        - 클래스 정보 : labels
    - 화폐당 하나의 클래스로 변경하세요.
        - json 파일에는 화폐 클래스가 앞뒷면으로 구분되어 있습니다.
        - 화폐의 앞뒷면 구분을 없애주세요.
            - 예시 : 'ten_front', 'ten_back' -> 'ten'
    - 화폐의 위치 정보를 YOLO 모델 형식에 맞게 변경 해주세요.
        - 사용되는 이미지는 원본에서 1/5로 축소되어 있습니다.
        - json 파일의 정보는 원본 기준 데이터이므로 위치 정보 추출을 할 때 x값과 y값을 1/5로 줄여주세요.
    - 이렇게 변경된 정보를 YOLO label 형식에 맞게 txt파일로 저장 해 주세요.
        - Hint : YOLO Labeling Format [label, x-center, y-center, width-norm, height-norm]
---

In [8]:
import json
import os

# 초기 경로 설정
json_path = './content/Dataset/labels/'
temp_list = ['train', 'val']

# 라벨을 숫자로 매핑하는 딕셔너리
label_dict = {
    'Ten': 0,
    'Fifty': 1,
    'Hundred': 2,
    'Five_Hundred': 3,
    'Thousand': 4,
    'Five_Thousand': 5,
    'Ten_Thousand': 6,
    'Fifty_Thousand': 7
}

# train과 val 폴더에 대해 반복
for folder in temp_list:
    folder_path = os.path.join(json_path, folder)
    json_files = [f for f in os.listdir(folder_path) if f.endswith('.json')]

    for json_file in json_files:
        with open(os.path.join(folder_path, json_file), 'r') as f:
            data = json.load(f)      
            
        # YOLO 포맷으로 정보 변환
        # x, y 값을 1/5로 줄임
        points = data['shapes'][0]['points']
        x1, y1 = points[0]
        x2, y2 = points[1]
        x1, y1, x2, y2 = x1 / 5, y1 / 5, x2 / 5, y2 / 5

        # 중심 좌표, 너비, 높이 계산
        x_center = (x1 + x2) / 2
        y_center = (y1 + y2) / 2
        width = x2 - x1
        height = y2 - y1

        # 이미지 너비와 높이
        img_width = data['imageWidth'] / 5
        img_height = data['imageHeight'] / 5

        # YOLO 포맷으로 normalize
        x_center /= img_width
        y_center /= img_height
        width /= img_width
        height /= img_height

        # 라벨 정보. 앞뒷면 구분을 없앰
        label_str = '_'.join(data['shapes'][0]['label'].rsplit('_', 1)[:-1])
        
        # 라벨을 숫자로 변환
        label_num = label_dict.get(label_str, -1)
        if label_num == -1:
            print(f"Unknown label: {label_str}")
            continue

        # YOLO 포맷으로 저장
        with open(os.path.join(folder_path, json_file.replace('.json', '.txt')), 'w') as f:
            f.write(f"{label_num} {x_center} {y_center} {width} {height}")


### (3) 데이터셋 정보가 담긴 파일 생성
---
- **세부요구사항**
    - 파일 안에 있어야 할 정보는 아래와 같습니다.
        - 학습할 클래스 이름 정보
        - 학습할 클래스 수 정보
        - Training, Validation 데이터셋 위치 정보
---
- 가장 대중적으로 이용하는 라이브러리는 yaml 입니다.
    - [yaml document](https://pyyaml.org/wiki/PyYAMLDocumentation)
    - 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
---

In [9]:
import yaml

# 데이터셋 정보를 담은 딕셔너리
dataset_info = {
    'names': ['10', '50', '100', '500', '1000', '5000', '10000', '50000'], # 학습할 클래스 이름 정보
    'nc': 8,  # 학습할 클래스 수 정보
    'train': '../content/Dataset/images/train',  # Training 데이터셋 위치 정보
    'val': '../content/Dataset/images/val'  # Validation 데이터셋 위치 정보
}

# yaml 파일로 저장
with open('./content/Dataset/money.yaml', 'w') as f:
    yaml.dump(dataset_info, f)

In [10]:
print(os.path.exists('./content/Dataset/money.yaml'))

True


## 3.모델링

### (1) 모델 라이브러리 설치
---

In [11]:
!git clone https://github.com/ultralytics/yolov5 
!pip install -r yolov5/requirements.txt

Cloning into 'yolov5'...


### (2) 가중치 파일 다운로드
---
- **세부요구사항**
    - 모델 개발자가 제공하는 사전 학습 가중치 파일을 다운로드 하세요.
        - 해당 과정이 불필요하다면 넘어가셔도 됩니다!
---

### (3) 학습 : train.py
---
- **세부요구사항**
    - UltraLytics YOLO v5에는 아래의 데이터가 필요합니다.
        - 데이터셋 정보가 담긴 yaml 파일
        - 사용하려는 모델 구조에 대한 yaml 파일
        - 사용하려는 모델의 가중치 파일
---

In [1]:
# %cd C:\Users\user\aivle\230921_MiniProject3
# !tensorboard --logdir runs/train

In [ ]:
%cd yolov5
!python train.py --img 640 --batch-size 16 --epochs 10 --data ../content/Dataset/money.yaml --cfg ./models/yolov5s.yaml --device 0

In [ ]:
%cd yolov5
!python train.py --img 640 --batch-size 16 --epochs 1000 --data ../content/Dataset/money.yaml --cfg ./models/yolov5s.yaml --device 0 --patience 3

## 4.탐지 : detect.py
---
- **세부요구사항**
    - 학습 과정에서 생성된 가중치 파일을 이용하세요.
    - IoU threshold를 0.25 이하로 설정하세요.
    - confidence threshold를 0.75 이상으로 설정하세요.
---
- 여러분이 **직접 촬영한 화폐 사진과 동영상**을 탐지 과정에 이용하여 결과를 확인하세요.
    - 조건
        1. 화폐의 수를 늘려가며 촬영 해보세요.
            - ex) 50원 하나, 50원 둘, 50원 셋, ...
        2. 화폐의 종류를 늘려가며 촬영 해보세요.
            - ex) 50원 하나와 100원 하나, 50원 하나와 100원 하나와 1000원 하나, ...
        3. 사진은 최소 30장 이상, 동영상은 최소 하나 이상 촬영하여 사용 해보세요.
---

In [ ]:
# 프로젝트 요구사항

In [ ]:
!python detect.py --weights runs/train/exp3/weights/best.pt --img 640 --conf 0.75 --iou 0.25 --source ./input_images/ --device 0 --save-txt --save-csv

In [ ]:
# GPT4 추천

In [ ]:
!python detect.py --weights runs/train/exp3/weights/best.pt --img 640 --conf 0.40 --iou 0.10 --source ./input_images/ --device 0 --save-txt --save-csv